# 🧑‍🍳 L4 - Frozen dinner: The design thinking meal

인벤토리:

1. 커널
2. 시맨틱(및 네이티브) 함수 - 이것으로 많은 일을 할 수 있습니다.
3. 비즈니스 사고 플러그인 --> 상상할 수 없는 방식의 SWOT
4. 디자인씽킹 플러그인 ... 여기 있습니다

# 🔥 Get a kernel ready

In [9]:
import os
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion
from IPython.display import display, Markdown

kernel = sk.Kernel()

useAzureOpenAI = False

if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_text_completion_service("azureopenai", AzureChatCompletion(deployment, endpoint, api_key))
else:
    # api_key, org_id = sk.openai_settings_from_dot_env()
    # kernel.add_text_completion_service("openai", OpenAIChatCompletion("gpt-3.5-turbo-0301", api_key, org_id))
    api_key = os.environ.get("OPENAI_API_KEY")
    kernel.add_text_completion_service("openai", OpenAIChatCompletion("gpt-3.5-turbo-0301", api_key))
    # kernel.add_text_completion_service("openai", OpenAIChatCompletion("gpt-4", api_key))  # write in Korean 이 적용되려면 gpt-4 필요

print("A kernel is now ready.")    

A kernel is now ready.


## 🏁 Let's start backwards from the customer

```directory
plugins-sk/
│
└─── DesignThinking/
     |
     └─── Define/
     |    └─── config.json
     |    └─── skprompt.txt
     |
     └─── Empathize/
          └─── config.json
          └─── skprompt.txt

```

In [15]:
import json

pluginsDirectory = "./plugins-sk"

strength_questions = ["피자 가게는 어떤 독특한 레시피나 재료를 사용합니까?","직원의 기술과 경험은 어떤가요?","피자 가게는 지역 내에서 평판이 좋은가요?","고객을 끌어들이는 가게 또는 위치의 독특한 특징이 있습니까?","피자 가게는 지역 내에서 평판이 좋은가요?","고객을 끌어들이는 가게 또는 위치의 독특한 특징이 있습니까?"]
weakness_questions = ["피자 가게의 운영상의 어려움은 무엇인가요? (예: 느린 서비스, 높은 직원 이직률)","성장이나 개선을 제한하는 재정적 제약이 있는가?","제품 제공에 부족한 점이 있는가?","해결해야 할 고객 불만이나 부정적인 리뷰가 있는가?"]
opportunities_questions = ["새로운 제품이나 서비스(예: 케이터링, 배달)에 대한 잠재력이 있는가?","서비스가 부족한 고객 세그먼트나 시장 영역이 있는가?","새로운 기술이나 시스템이 비즈니스 운영을 개선할 수 있는가?","마케팅에 활용할 수 있는 파트너십이나 지역 이벤트가 있는가?"]
threats_questions = ["주요 경쟁자는 누구이며 무엇을 제공하는가?","지역 변화(예: 공사, 인근 사업장 폐쇄)로 인한 잠재적 부정적 영향이 있는가?","사업에 부정적인 영향을 미칠 수 있는 경제 또는 산업 동향(예: 재료비 상승)이 있는가?","규제 또는 법률 변경으로 인한 위험(예: 건강 및 안전, 고용)이 있는가?"]

strengths = [ "최고 상을 수상한 독특한 마늘 피자 레시피","시칠리아에서 훈련받은 소유주","강력한 지역 평판","대학 캠퍼스의 주요 위치", "대학 캠퍼스의 주요 위치" ]
weaknesses = [ "높은 직원 이직률", "홍수로 인해 수리가 필요한 좌석 공간이 손상됨", "메뉴에서 인기 있는 칼초네가 없음", "최신 유행 재료가 부족하다는 젊은 층의 부정적인 리뷰" ]
opportunities = [ "미개척 케이터링 잠재력", "성장하는 지역 기술 스타트업 커뮤니티", "미개척 온라인 존재 및 주문 기능", "다가오는 연례 음식 박람회", "미개척 온라인 존재 및 주문 기능"]
threats = [ "인근의 저렴한 피자 업체와의 경쟁","유동 인구에 영향을 미칠 인근 도로 공사가 진행 중","치즈 가격 상승으로 피자 가격이 상승할 것","즉각적인 지역 규제 변화는 없지만 선거 시즌" ]

customer_comments = """
고객 1: 좌석이 정말 거칠어 보여요.
고객 2: 마늘 피자는 지구상에서 최고예요.
고객 3: 방문할 때마다 서버가 바뀐다는 걸 알았는데, 전혀 모르겠어요.
고객 4: 칼조네는 왜 없나요?
고객 5: 마늘 피자를 좋아하는데 다른 곳에서는 구할 수 없어요.
고객 6: 마늘 피자는 정말 맛있어요.
고객 7: 피자에 비해 휴대하기 편한 칼초네를 선호합니다.
고객 8: 피자가 왜 이렇게 비싼가요?
고객 9: 온라인 주문이 불가능해요.
고객 10: 좌석이 왜 이렇게 불편하고 더러운가요?
"""

# strength_questions = ["What unique recipes or ingredients does the pizza shop use?","What are the skills and experience of the staff?","Does the pizza shop have a strong reputation in the local area?","Are there any unique features of the shop or its location that attract customers?", "Does the pizza shop have a strong reputation in the local area?", "Are there any unique features of the shop or its location that attract customers?"]
# weakness_questions = ["What are the operational challenges of the pizza shop? (e.g., slow service, high staff turnover)","Are there financial constraints that limit growth or improvements?","Are there any gaps in the product offering?","Are there customer complaints or negative reviews that need to be addressed?"]
# opportunities_questions = ["Is there potential for new products or services (e.g., catering, delivery)?","Are there under-served customer segments or market areas?","Can new technologies or systems enhance the business operations?","Are there partnerships or local events that can be leveraged for marketing?"]
# threats_questions = ["Who are the major competitors and what are they offering?","Are there potential negative impacts due to changes in the local area (e.g., construction, closure of nearby businesses)?","Are there economic or industry trends that could impact the business negatively (e.g., increased ingredient costs)?","Is there any risk due to changes in regulations or legislation (e.g., health and safety, employment)?"]

# strengths = [ "Unique garlic pizza recipe that wins top awards","Owner trained in Sicily","Strong local reputation","Prime location on university campus" ]
# weaknesses = [ "High staff turnover","Floods in the area damaged the seating areas that are in need of repair","Absence of popular calzones from menu","Negative reviews from younger demographic for lack of hip ingredients" ]
# opportunities = [ "Untapped catering potential","Growing local tech startup community","Unexplored online presence and order capabilities","Upcoming annual food fair" ]
# threats = [ "Competition from cheaper pizza businesses nearby","There's nearby street construction that will impact foot traffic","Rising cost of cheese will increase the cost of pizzas","No immediate local regulatory changes but it's election season" ]

# customer_comments = """
# Customer 1: The seats look really raggedy.
# Customer 2: The garlic pizza is the best on this earth.
# Customer 3: I've noticed that there's a new server every time I visit, and they're clueless.
# Customer 4: Why aren't there calzones?
# Customer 5: I love the garlic pizza and can't get it anywhere else.
# Customer 6: The garlic pizza is exceptional.
# Customer 7: I prefer a calzone's portable nature as compared with pizza.
# Customer 8: Why is the pizza so expensive?
# Customer 9: There's no way to do online ordering.
# Customer 10: Why is the seating so uncomfortable and dirty?
# """

pluginDT = kernel.import_semantic_skill_from_directory(pluginsDirectory, "DesignThinking");
my_result = await kernel.run_async(pluginDT["Empathize"], input_str=customer_comments)

# display(Markdown("## ✨ The categorized observations from the 'Empathize' phase of design thinking\n"))
display(Markdown("## ✨ 디자인 사고의 '공감' 단계에서 분류된 관찰 결과\n"))

print(my_result)
# print(json.dumps(json.loads(str(my_result)), indent=2))


## ✨ 디자인 사고의 '공감' 단계에서 분류된 관찰 결과


[
            { "sentiment": "맛이 좋다", 
              "summary": "마늘 피자가 맛있다는 의견이 여러 번 나왔습니다."},
            { "sentiment": "불편하다", 
              "summary": "좌석이나 칼조네의 불편함에 대한 의견이 여러 번 나왔습니다."},
            { "sentiment": "서비스에 대한 불만", 
              "summary": "서버의 불안정성이나 온라인 주문 불가능에 대한 불만이 여러 번 나왔습니다."},
            { "sentiment": "가격에 대한 불만", 
              "summary": "피자의 비싼 가격에 대한 불만이 나왔습니다."},
            { "sentiment": "다른 곳에서 구하기 어렵다", 
              "summary": "마늘 피자를 다른 곳에서 구하기 어렵다는 의견이 나왔습니다."}
        ]


In [3]:
print(str(my_result))
# print(json.loads(str(my_result)))

[
            { "sentiment": "불만족", 
              "summary": "좌석이 낡아 보인다."},
            { "sentiment": "만족", 
              "summary": "맛있는 마늘 피자."},
            { "sentiment": "불만족", 
              "summary": "매번 새로운 서버가 있고, 무지합니다."},
            { "sentiment": "불만족", 
              "summary": "칼초네가 없어서 실망합니다."},
            { "sentiment": "만족", 
              "summary": "맛있는 마늘 피자를 다른 곳에서는 못 먹어서 좋아합니다."}
        ]


![](./assets/designthinking.png)


**Note:** LLM이 매번 동일한 결과를 산출하는 것은 아닙니다. 결과는 비디오와 다를 수 있습니다.

In [17]:
my_result = await kernel.run_async(pluginDT["Empathize"], pluginDT["Define"], input_str = customer_comments)

# display(Markdown("## ✨ The categorized observations from the 'Empathize' + 'Define' phases of design thinking\n"+str(my_result)))
display(Markdown("## ✨ 디자인 사고의 '공감하기' + '정의하기' 단계에서 분류된 관찰 결과\n"+str(my_result)))

## ✨ 디자인 사고의 '공감하기' + '정의하기' 단계에서 분류된 관찰 결과
| 분석 | 정의 | 가능한 원인 |
| --- | --- | --- |
| 맛 좋음 | 마늘 피자가 맛있어서 좋아함 | 좋은 음식 경험 |
| 서비스 불만족 | 서버가 자주 바뀌어서 불편함 | 부적절한 서비스 제공 |
| 메뉴 불만족 | 칼조네가 없어서 불만족함 | 부적절한 메뉴 선택 |
| 가격 불만족 | 피자 가격이 비싸서 불만족함 | 부적절한 가격 책정 |
| 시설 불만족 | 좌석이 불편하고 더러워서 불만족함 | 부적절한 시설 유지보수 |

In [19]:
my_result = await kernel.run_async(pluginDT["Empathize"], pluginDT["Define"], pluginDT["Ideate"], pluginDT["PrototypeWithPaper"], input_str=customer_comments)

# display(Markdown("## ✨ The categorized observations from the 'Empathize' + 'Define' + 'Ideate' + 'Prototype' + phases of design thinking\n"+str(my_result)))
display(Markdown("## ✨ '공감하기' + '정의하기' + '아이디어 내기' + '프로토타입 만들기' + 디자인 사고의 단계에서 분류된 관찰 결과입니다.\n"+str(my_result)) )

## ✨ '공감하기' + '정의하기' + '아이디어 내기' + '프로토타입 만들기' + 디자인 사고의 단계에서 분류된 관찰 결과입니다.
1. 온라인 주문 시스템 도입
- 종이에 피자 주문 화면을 그려서 주문하는 과정을 시뮬레이션한다.
- 주문 시스템을 사용하는 고객과 직원 역할을 나누어 연습한다.

2. 고객 리뷰를 활용한 개선점 도출
- 종이에 고객 리뷰를 적고, 문제점과 개선점을 색으로 구분한다.
- 직원들끼리 리뷰를 공유하고, 문제점을 해결하는 방법을 토론한다.

3. 주간 할인 이벤트 진행
- 종이에 할인 쿠폰을 만들어서 고객에게 제공한다.
- 할인 쿠폰을 사용하는 고객과 직원 역할을 나누어 연습한다.

4. 피자 포장용 밀봉기 구매
- 종이에 밀봉기를 그려서 포장하는 과정을 시뮬레이션한다.
- 포장하는 과정에서 발생할 수 있는 문제점을 파악하고, 해결 방법을 찾는다.

5. 주문 후 픽업 시간 예약 기능 추가
- 종이에 주문 예약 화면을 그려서 시뮬레이션한다.
- 예약하는 고객과 직원 역할을 나누어 연습한다.

6. 음식 사진 및 설명 추가
- 종이에 메뉴판을 그려서 사진과 설명을 추가한다.
- 고객이 주문할 때 메뉴판을 보면서 주문하는 과정을 시뮬레이션한다.

7. 고객 생일 이벤트 진행
- 종이에 생일 이벤트 쿠폰을 만들어서 고객에게 제공한다.
- 이벤트 쿠폰을 사용하는 고객과 직원 역할을 나누어 연습한다.

8. 음식 재료 원산지 공개
- 종이에 메뉴판을 그려서 재료 원산지를 추가한다.
- 고객이 주문할 때 메뉴판을 보면서 주문하는 과정을 시뮬레이션한다.

9. 피자 크기 선택 기능 추가
- 종이에 주문 화면을 그려서 크기 선택 기능을 추가한다.
- 주문하는 고객과 직원 역할을 나누어 연습한다.

10. 포장용 수저, 포크 제공
- 종이에 포장용 수저와 포크를 그려서 포장하는 과정을 시뮬레이션한다.
- 포장하는 과정에서 발생할 수 있는 문제점을 파악하고, 해결 방법을 찾는다.

In [27]:
# sk_prompt = """
# 직장에서 막 근무를 마치고 바에 들어온 40세 남성이 있습니다. 기분이 좋지 않습니다.

# 이 남성에게 다음과 같은 경험이 주어집니다:
# {{$input}}

# 이 경험에 대한 그들의 가능한 반응을 요약하세요. 
# """

sk_prompt = """
A 40-year old man who has just finished his shift at work and comes into the bar. They are in a bad mood.

They are given an experience like:
{{$input}}

Summarize their possible reactions to this experience. 
You must write in Korean.
"""
test_function = kernel.create_semantic_function(prompt_template=sk_prompt,
                                                    description="Simulates reaction to an experience.",
                                                    max_tokens=1000,
                                                    temperature=0.1,
                                                    top_p=0.5)
sk_input="""
각 로열티 레벨에 따른 리워드, 포인트 적립 방법, 리워드 사용 방법 등의 세부 정보가 포함된 간단한 로열티 카드가 바를 방문하는 모든 분께 제공됩니다.
"""

# sk_input="""
# A simple loyalty card that includes details such as the rewards for each level of loyalty, how to earn points, and how to redeem rewards is given to every person visiting the bar.
# """

test_result = await kernel.run_async(test_function, input_str=sk_input) 

display(Markdown("### ✨ " + str(test_result)))

### ✨ 40대 남성이 일을 마치고 바에 들어오는데 기분이 좋지 않습니다. 그러나 바에서 제공하는 로열티 카드에는 각 로열티 레벨에 따른 리워드, 포인트 적립 방법, 리워드 사용 방법 등의 세부 정보가 포함되어 있습니다. 이에 대한 가능한 반응은 다음과 같습니다.

1. 기분이 좋아집니다: 로열티 카드를 받고 세부 정보를 확인하면, 기분이 좋아질 수 있습니다. 더 많은 리워드와 혜택을 받을 수 있다는 것을 알게 되면, 기분이 좋아질 수 있습니다.

2. 놀랍습니다: 로열티 카드에 포함된 세부 정보가 매우 상세하고 다양하다면, 이에 대해 놀랄 수 있습니다. 이전에는 로열티 프로그램에 대해 잘 몰랐거나, 이정도의 혜택을 받을 수 있다는 것을 몰랐을 수도 있습니다.

3. 관심이 생깁니다: 로열티 카드를 받고 세부 정보를 확인하면, 로열티 프로그램에 대한 관심이 생길 수 있습니다. 더 많은 혜택을 받기 위해 로열티 프로그램에 참여하고, 포인트를 적립하려는 생각이 들 수 있습니다.

4. 불만스러워합니다: 로열티 카드를 받아도, 기분이 좋아지지 않는 경우도 있습니다. 예를 들어, 로열티 프로그램의 혜택이 별로 없거나, 적립된 포인트를 사용하기가 어렵다면, 불만스러워할 수 있습니다.

5. 무관심합니다: 로열티 카드를 받아도, 로열티 프로그램에 대해 별로 관심이 없는 경우도 있습니다. 이 경우에는 로열티 카드를 받아도, 특별한 반응이 없을 수 있습니다.

## 🔖 알림: 아직 명시적으로 🧲 유사성 엔진을 사용하지 않았지만, 다음에는 사용할 예정입니다! 

![](./assets/twodimensions.png)